In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

url_base = 'http://www.chicagomag.com'
url_sub = '/Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/'
url = url_base + url_sub

html = urlopen(url)
soup = BeautifulSoup(html, 'html.parser')

soup

In [ ]:
print(soup.find_all('div', 'sammy'))

In [ ]:
len(soup.find_all('div', 'sammy'))

In [ ]:
tmp_one = soup.find_all('div', 'sammy')[0]
type(tmp_one)

In [ ]:
tmp_one.find(class_='sammyRank')

In [ ]:
tmp_one.find(class_='sammyRank').get_text()

In [ ]:
tmp_one.find(class_='sammyListing').get_text()

In [ ]:
tmp_one.find('a')['href']

In [ ]:
import re
tmp_string = tmp_one.find(class_='sammyListing').get_text()
re.split(('\n|\r\n'), tmp_string)

print(re.split(('\n|\r\n'), tmp_string)[0])
print(re.split(('\n|\r\n'), tmp_string)[1])

In [ ]:
from urllib.request import urljoin

rank = []
main_menu = []
cafe_name = []
url_add = []

list_soup = soup.find_all('div', 'sammy')

for item in list_soup:
    rank.append(item.find(class_='sammyRank').get_text())
    
    tmp_string = item.find(class_='sammyListing').get_text()
    
    main_menu.append(re.split(('\n|\r\n'), tmp_string)[0])
    cafe_name.append(re.split(('\n|\r\n'), tmp_string)[1])
    
    url_add.append(urljoin(url, item.find('a')['href']))

In [ ]:
rank[:5]

In [ ]:
main_menu[:5]

In [ ]:
cafe_name[:5]

In [ ]:
url_add[:5]

In [ ]:
import pandas as pd
data = {'Rank': rank, 'Menu': main_menu, 'Cafe': cafe_name, 'URL':url_add}
df = pd.DataFrame(data)
df.head()

In [ ]:
df = pd.DataFrame(data, columns=['Rank', 'Cafe', 'Menu', 'URL'])

In [ ]:
df.to_csv('../data/03.base_sandwiches_list_chicago.csv', sep=',',
          encoding='UTF-8')

In [ ]:
df = pd.read_csv('../data/03.base_sandwiches_list_chicago.csv', index_col=0)
df.head()

In [ ]:
html = urlopen(df['URL'][0])
soup_tmp = BeautifulSoup(html, 'html.parser')
soup_tmp

In [ ]:
print(soup_tmp.find('p', 'addy'))

In [ ]:
price = []
address = []

for n in df.index[:3]:
    html = urlopen(df['URL'][n])
    soup_tmp = BeautifulSoup(html, 'html.parser')
    gettings = soup_tmp.find('p', 'addy').get_text()
    price.append(gettings.split()[0][:-1])
    address.append(' '.join(gettings.split()[1:-2]))

In [ ]:
price

In [ ]:
address

In [ ]:
from tqdm import tqdm_notebook
price = []
address = []

for n in tqdm_notebook(df.index):
    html = urlopen(df['URL'][n])
    soup_tmp = BeautifulSoup(html, 'html.parser')
    gettings = soup_tmp.find('p', 'addy').get_text()
    price.append(gettings.split()[0][:-1])
    address.append(' '.join(gettings.split()[1:-2]))

In [ ]:
price

In [ ]:
address

In [ ]:
df['Price'] = price
df['Address'] = address

In [ ]:
df.head()

In [ ]:
df = df.loc[:, ['Rank', 'Cafe', 'Menu', 'Price', 'Address']]
df.set_index('Rank', inplace=True)
df.head()

In [ ]:
df.to_csv('../data/03.best_sandwiches_list_chicago2.csv',
          sep=',', encoding='UTF-8')

In [ ]:
import folium
import pandas as pd
import googlemaps
import numpy as np

df = pd.read_csv('../data/03.best_sandwiches_list_chicago2.csv', index_col=0)
df.head(5)

In [ ]:
gmaps_key = '...'
gmaps = googlemaps.Client(key=gmaps_key)

In [ ]:
lat = []
lng = []

for n in tqdm_notebook(df.index):
    if df['Address'][n] != 'Multiple':
        target_name = df['Address'][n] + ', ' + 'Cicago'
        gmaps_output = gmaps.geocode(target_name)
        location_output = gmaps_output[0].get('geometry')
        lat.append(location_output['location']['lat'])
        lng.append(location_output['location']['lng'])
    else:
        lat.append(np.nan)
        lng.append(np.nan)

In [ ]:
df['lat'] = lat
df['lng'] = lng

In [ ]:
df.head()

In [ ]:
mapping = folium.Map(location=[df['lat'].mean(), df['lng'].mean()],
                     zoom_start=11)
folium.Marker([df['lat'].mean(), df['lng'].mean()],
              popup='center').add_to(mapping)
mapping

In [ ]:
df.index

In [ ]:
mapping = folium.Map(location=[df['lat'].mean(), df['lng'].mean()],
                     zoom_start=11)
for n in range(1, 50):
    if df['Address'][n] != 'Multiple':
        folium.Marker([df['lat'][n], df['lng'][n]]
                      , popup=df['Cafe'][n],
                      ).add_to(mapping)
mapping